In [3]:
# import sqlite3 as db
# import re
import pymysql
# import sys
# import time
# import configparser
# import pprint
# import json
import csv
# import os


### 1. sqlite抓所有爬蟲下來的資料 
- tbl <- readDB(dbName)
 
### 2. 拆欄位 
- urls, nosqls, rdbs, titles, mains, details, getTimes <- splitColumnFromDataSet(tbl)
 
###  3. 逐筆清洗(欄位解析)        
- url, label, rent, lat, lng, sex, space, smoke, pet, cook, parking, updateDate <- Extracter_sy(self, url, title, main, detail, getTime)

### 4. 逐筆匯入 SQL
- upsertHouse(a.getUrl, a.getLabel, str(a.getRent), a.getLat, a.getLng, a.getSex, str(a.getSpace), a.getSmoke, a.getPet, a.getCook, a.getParking, a.getDate)

### 5. 逐筆匯入 mongoFormatJSON
- writeMongoFormat(houseObj), writeMongoFormat(houseObj)

In [2]:
# 永慶房屋, 信義房屋, 中信房屋, 好房網
def readDB(target):
    with db.connect(target) as conn:
        c = conn.cursor()
        c.execute('select url, title, main, detail, getTime from rental;')
        result = c.fetchall()
    conn.close()
    return result
#

In [3]:
# 住商不動產
def readDB_lb(target):
    with db.connect(target) as conn:
        c = conn.cursor()
        c.execute("select url, title, address, pattern, ttype, avg_rent, lat, lng, sex, space, smoke, pet, cook, parking, updateDate FROM rental;")
        # c.execute("select url, floor, stories, title, address, pattern, label, rent, lat, lng, sex, space, smoke, pet, cook, parking, updateDate FROM rental;")
        result = c.fetchall()
    conn.close()
    return result
#

In [4]:
# 21世紀、太平洋房屋  (csv 格式檔案)
def readDB_csv(target):
    tbl = []
    with open(target, encoding='utf8') as f:
        for row in csv.reader(f):
            tbl.append(row)
        return tbl
#

In [5]:
# 591租屋、好房網  (json格式)
def readDB_json(target):
    with open(target, encoding='utf8') as f:
        return json.load(f)
#

In [6]:
# 永慶房屋, 信義房屋, 中信房屋, 好房網    db欄位的拆解方法
def splitColumnFromDataSet(tbl):
    urls = []; titles = []; mains = []; details = []; getTimes = []; 
    for r in tbl:
        if r[0] != None:
            urls.append(r[0])
            titles.append(r[1])
            mains.append(r[2])
            details.append(r[3])
            getTimes.append(r[4])
    return urls, titles, mains, details, getTimes
#

In [7]:
# 住商不動產  db欄位拆解方法
def splitColumnFromDataSet_lb(tbl):
    urls = [];        titles = [];     addresss = [];     patterns = [];     labels = [];      floors = []; 
    rents = [];       lats = [];       lngs = [];         sexs = [];         spaces = [];      storiess = []; 
    smokes = [];      pets = [];       cooks = [];        parkings = [];     updateDates = []
    for r in tbl:
        if r[0] != None:
            urls.append(r[0])
            titles.append(r[1])
            addresss.append(r[2])
            patterns.append(r[3])
            labels.append(r[4])
            rents.append(r[5])
            lats.append(r[6])
            lngs.append(r[7])
            sexs.append(r[8])
            spaces.append(r[9])
            smokes.append(r[10])
            pets.append(r[11])
            cooks.append(r[12])
            parkings.append(r[13])
            updateDates.append(r[14])
            storiess.append(r[15])
            floors.append(r[16])

    return urls, titles, addresss, patterns, labels, rents, lats, lngs, sexs, spaces, smokes, pets, cooks, parkings, updateDates, storiess, floors
#

In [8]:
#  好房網 json欄位拆解方法
def splitColumnFromDataSet_hf(tbl):
    urls = []; titles = []; addresss = []; patterns = []; uses = []; rents = []; sexs = []; 
    spaces = []; pets = []; cooks = []; parkings = []; updateDates = []; floors = []
    for r in tbl:
        urls.append(tbl[r]['網址'])
        titles.append(tbl[r]['標題'])
        addresss.append(tbl[r]['地址'])
        patterns.append(tbl[r]['格局'])
        floors.append(tbl[r]['樓層'])
        rents.append(tbl[r]['租金'])
        sexs.append(tbl[r]['性別限制'])
        spaces.append(tbl[r]['坪數'])
        pets.append(tbl[r]['寵物'])
        cooks.append(tbl[r]['開火'])
        parkings.append(tbl[r]['車位'])
        updateDates.append(tbl[r]['紀錄時間'])
    return urls, titles, addresss, patterns, floors, rents, sexs, spaces, pets, cooks, parkings, updateDates
#

In [9]:
# 591租屋  json欄位拆解方法 (修改前)
def splitColumnFromDataSet_591(tbl):
    urls = []; rents = []; addresses = []; contexts = []; titles = []; patterns = []; getTimes = [];
    for r in tbl:
        urls.append(r)
        rents.append(tbl[r]['price'])
        addresses.append(tbl[r]['address'])
        contexts.append(tbl[r]['context'])
        titles.append(tbl[r]['title'])
        patterns.append(tbl[r]['pattern'])
        getTimes.append(tbl[r]['getTime'])
    return urls, rents, addresses, contexts, titles, patterns, getTimes
#

In [10]:
# # 好房網  json欄位拆解方法 (修改前)
# def splitColumnFromDataSet_hf(tbl):
#     urls = []; titles = []; addresss = []; patterns = []; uses = []; rents = []; sexs = []; 
#     spaces = []; pets = []; cooks = []; parkings = []; getTimes = []; floors = []
#     for r in tbl:
#         urls.append(tbl[r]['網址'])
#         titles.append(tbl[r]['標題'])
#         addresss.append(tbl[r]['地址'])
#         patterns.append(tbl[r]['格局'])
#         floors.append(tbl[r]['樓層'])
#         rents.append(tbl[r]['租金'])
#         sexs.append(tbl[r]['性別限制'])
#         spaces.append(tbl[r]['坪數'])
#         pets.append(tbl[r]['寵物'])
#         cooks.append(tbl[r]['開火'])
#         parkings.append(tbl[r]['車位'])
#         getTimes.append(tbl[r]['紀錄時間'])
#     return urls, titles, addresss, patterns, floors, rents, sexs, spaces, pets, cooks, parkings, getTimes
# #

In [11]:
# 讀取外部組態檔
def getConfig(cfgini):
    config = configparser.ConfigParser()
    config.read(cfgini)
    dbhost = config.get('Section_A', 'sql_dbhost')
    user = config.get('Section_A', 'sql_id')
    passwd = config.get('Section_A', 'sql_pd')
    return dbhost, user, passwd
#

In [12]:
# 更新修改 mySQL資料庫
def upsertHouse(h, dbhost, user, passwd):
    try:
        db = pymysql.connect(host = dbhost, port = 3306, user = user, passwd = passwd, db = 'house', charset='utf8mb4')
        c = db.cursor()
        c.execute("SELECT url FROM rental WHERE url = %s;", (h.getUrl,))
        if (len(c.fetchall()) == 0):
            job = 'insert'
            doing = "INSERT INTO rental(url, title, address, pattern, floor, stories, label, rent, lat, lng, sex, space, smoke, pet, cook, parking, updateDate) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
            vv = (h.getUrl, h.getTitle, h.getAddress, h.getPattern, str(h.getFloor), str(h.getStories), h.getLabel, str(h.getRent), h.getLat, h.getLng, h.getSex, str(h.getSpace), h.getSmoke, h.getPet, h.getCook, h.getParking, h.getDate)

            c.execute(doing, vv)
            db.commit()

        else:
            job = 'update'
            doing = "UPDATE rental SET updateDate = %s  WHERE url = %s ;"
            vv = (h.getDate, h.getUrl)

            c.execute(doing, vv)
            db.commit()

    except Exception as err:
        print('error: upsertHouse() with ' + h.getUrl + ' on ' + str(h.getDate) + ' when ' + job) 
        print(str(err))

    finally:
        c.close()
        db.close()
#

In [13]:
# 信義房屋_建彰
class Extracter_sy():
    pass

    def __init__(self, url, title, main, detail, getTime):
        self.__url = url
        self.__title = title
        self.__main = main
        self.__detail = detail
        self.__getTime = getTime
        self.__YN = True
        
    @property # 調整的儲存程序
    def getYN(self):
        return self.__YN
        
    @property
    def getUrl(self):                            # OK
        return self.__url
    
    @property
    def getTitle(self):                          # OK
        return self.__title
    
    @property
    def getDate(self):
        d = self.__getTime.split(' ')[0]
        fmt = '%Y-%m-%d'
        return time.strptime(d, fmt)
    
    @property
    def getLabel(self):
        reg = re.compile(r'^[1-9][0-9]*房')      #  需要大量debug
        
        if self.__detail['類型'] == '店面':
            return 'S'
        elif self.__detail['類型'] == '住辦':
            return 'O'
        elif self.__detail['類型'] in ['透天厝/別墅', '雅房', '套房', '整層住宅']:
            return 'H'
        elif self.__detail['類型'] in ['辦公/廠房', '車位', '其它', '土地/場地']:
            return 'F'
        else:
            return 'X'
    
    @property
    def getFloor(self): # 所在樓層 
        return self.__floor('f')
    
    @property
    def getStories(self): # 總樓層 
        return self.__floor('t')

    def __floor(self, ftype):
        try:
            if self.__main.get('floor'):
                reg = re.compile(r'-?[\d]+') 
                if '/' in self.__main['floor']:
                    floorString = self.__main['floor'].split('/')[0] if ftype == 'f' else self.__main['floor'].split('/')[1]
                    floor = reg.search(floorString).group()

                elif self.__main['floor'] == '0樓':
                    floorString = self.__main['floor']
                    floor = reg.search(floorString).group()
                    
                floor = int(floor)
                if floor == 0:
                    return 1
                else:
                    return floor
            
        except Exception as err:
            return 0
    
    @property
    def getPattern(self):                        # OK
        if self.__detail['格局']:
            return self.__detail['格局']
        return ''
    
    @property
    def getAddress(self):                        # OK
        if self.__detail['地址']:
            return self.__detail['地址'].replace('　','')
        return ''
    
    @property
    def getRent(self):                       # 需要大量debug
        reg = re.compile(r'([0-9]{1,3},?[0-9]{3})+')
        if reg.search(self.__main['rent']):
            mo = reg.match(self.__main['rent'])
            return int(mo.group().replace(',', ''))
        return -1
    
    @property
    def getLat(self):                            # OK
        return self.__main['latlng'].split(',')[0]
        
    
    @property
    def getLng(self):                            # OK
        return self.__main['latlng'].split(',')[1]
    
    @property
    def getSex(self):                            # 需要大量debug
        if self.__detail.get('身份限制'):
            if '不拘' in self.__detail['身份限制']:
                return 'B'
            elif '限男性' in self.__detail['身份限制']:
                return 'M'
            elif '限女性' in self.__detail['身份限制']:
                return 'F'
        return ''

    @property
    def getSpace(self):                          # 需要大量debug
        if self.__detail.get('建物坪數'):
            mo = re.match(r'\d+(.[\d]+)?', self.__detail['建物坪數'])
            sp = float(mo.group())
            if sp > 250:         # 坪數太大者, 直接忽略
                self.__YN = False
            return sp
        else:
            return -1
    
    @property
    def getSmoke(self):                          # 需要大量debug
        if self.__detail.get('生活約定'):
            if '抽菸不可' in self.__detail['生活約定']:
                return 'N'
            elif '抽菸可' in self.__detail['生活約定']:
                return 'Y'
        else:
            return ''
    
    @property
    def getPet(self):                            # 需要大量debug
        if self.__detail.get('生活約定'):
            if '養寵物不可' in self.__detail['生活約定']:
                return 'N'
            elif '養寵物可' in self.__detail['生活約定']:
                return 'Y'
        else:
            return ''
    
    @property
    def getCook(self):                           # 需要大量debug
        if self.__detail.get('生活約定'):
            if '開伙不可' in self.__detail['生活約定']:
                return 'N'
            elif '開伙可' in self.__detail['生活約定']:
                return 'Y'
        else:
            return ''
    
    @property
    def getParking(self):                        # 需要大量debug
        
        if self.__detail.get('車位'):
            if '年' in self.__detail['車位']:
                self.__YN = False
                return 'X'
            if '無' == self.__detail['車位']:
                return 'N'
            elif self.__detail['車位'] in ['其他', '另租', '坡道平面', '坡道機械', '庭院', '有', '機械平面']:
                return 'Y'
        else:
            return ''
#

In [14]:
# 中信房屋_建彰
class Extracter_jc():
    pass

    def __init__(self, url, title, main, detail, getTime):
        self.__url = url
        self.__title = title
        self.__main = main
        self.__detail = detail
        self.__getTime = getTime
        self.__YN = True
        
    @property # 調整的儲存程序
    def getYN(self):
        return self.__YN
        
    @property
    def getUrl(self):                            # OK
        return self.__url
    
    
    @property
    def getFloor(self): # 所在樓層 
        return self.__floor('f')
    
    @property
    def getStories(self): # 總樓層 
        return self.__floor('t')

    def __floor(self, ftype):
        try:
            if self.__main.get('樓層'):
                reg = re.compile(r'-?[\d]+') 
                
                if self.__main['樓層'] == None:
                    return 0
                
                elif '/' in self.__main['樓層']:
                    floorString = self.__main['樓層'].split('/')[0] if ftype == 'f' else self.__main['樓層'].split('/')[1]
                    floor = reg.search(floorString).group()
                    
                elif self.__main['樓層'] == '0樓':
                    floorString = self.__main['樓層']
                    floor = reg.search(floorString).group()
                    
                floor = int(floor)
                
                if floor == 0:
                    return 1
                    
                else:
                    return floor
            else:
                return 0
            
        except Exception as err:
            return 0
    
    @property
    def getTitle(self):                          # OK
        return self.__title
    
    @property
    def getDate(self):
        d = self.__getTime.split(' ')[0]
        fmt = '%Y-%m-%d'
        return time.strptime(d, fmt)
    
    @property
    def getLabel(self):
        ttype = self.__main['類型']
        if '店面' in ttype:
            return 'S'
        
        if '辦公' in ttype or '住辦' in ttype:
            return 'O'
        
        if '獨立套房' in ttype or '雅房' in ttype or '分租套房' in ttype:
            return 'H'
        
        else:
            return 'F'

    @property
    def getPattern(self):                        # OK
        if self.__main.get('格局'):
            return self.__main['格局']
        return ''
    
    
    @property
    def getAddress(self):                        # OK
        if self.__main['地址']:
            return self.__main['地址'].replace('　','')
        return ''
    
    @property
    def getRent(self):                       # 需要大量debug
        
        reg = re.compile(r'(\d)+')
        if reg.search(self.__main['租金']):
            mo = reg.search(self.__main['租金'])
            return int(mo.group().replace(',', ''))
        return -1
    
    @property
    def getLat(self):                            # OK
        if self.__main.get('lat'):
            return self.__main['lat']
        else:
            return -1
    
    @property
    def getLng(self):                            # OK
        if self.__main.get('lng'):
            return self.__main['lng']
        else:
            return -1
    
    @property
    def getSex(self):                            # 需要大量debug
        if self.__detail.get('性別要求'):
            if '男女皆可' in self.__detail['性別要求']:
                return 'B'
            elif '男' in self.__detail['性別要求']:
                return 'M'
            elif '女' in self.__detail['性別要求']:
                return 'F'
        return ''

    @property
    def getSpace(self):                          # 需要大量debug
        if self.__main.get('坪數'):
            mo = re.match(r'\d+(.[\d]+)?', self.__main['坪數'])
            sp = float(mo.group())
            if sp > 250:         # 坪數太大者, 直接忽略
                self.__YN = False
            return sp
        
        else:
            return -1
    
    @property
    def getSmoke(self):                          # 需要大量debug
        if self.__detail.get('生活約定'):
            if '抽菸不可' in self.__detail['生活約定']:
                return 'N'
            elif '抽菸可' in self.__detail['生活約定']:
                return 'Y'
        else:
            return ''
    
    @property
    def getPet(self):                            # 需要大量debug
        if self.__detail.get('寵物'):
            if '不接受' in self.__detail['寵物']:
                return 'N'
            elif '接受' in self.__detail['寵物']:
                return 'Y'
        else:
            return ''
    
    @property
    def getCook(self):                           # 需要大量debug
        if self.__detail.get('開伙'):
            if '不接受' in self.__detail['開伙']:
                return 'N'
            elif '接受' in self.__detail['開伙']:
                return 'Y'
        else:
            return ''
    
    @property
    def getParking(self):                        # 需要大量debug
        if self.__main.get('車位'):
            if self.__main['車位'] in ['租金內不含車位', '無']:
                return 'N'
            elif self.__main['車位'] in ['租金內含車位', '有車位']:
                return 'Y'
            {'', '', '', ''}
        else:
            return ''
# 中信房屋

In [15]:
# 永慶房屋
class Extracter_yc():
    pass

    def __init__(self, url, title, main, detail, getTime):
        self.__url = url
        self.__title = title
        self.__main = main
        self.__detail = detail
        self.__getTime = getTime
        self.__YN = True
        
    @property # 黑箱程序
    def getYN(self):
        return self.__YN
        
    @property
    def getUrl(self):                            # OK
        return self.__url
    
    @property
    def getFloor(self): # 所在樓層 
        return self.__floor('f')
    
    @property
    def getStories(self): # 總樓層 
        return self.__floor('t')

    def __floor(self, ftype):
        try:
            if self.__main.get('樓別/樓高'):
                reg = re.compile(r'-?[\d]+') 
                
                if self.__main['樓別/樓高'] == None:
                    return 0
                
                elif '/' in self.__main['樓別/樓高']:
                    floorString = self.__main['樓別/樓高'].split('/')[0] if ftype == 'f' else self.__main['樓別/樓高'].split('/')[1]
                    floor = reg.search(floorString).group()
                    
                elif self.__main['樓別/樓高'] == '0樓':
                    floorString = self.__main['樓別/樓高']
                    floor = reg.search(floorString).group()
                    
                floor = int(floor)
                
                if floor == 0:
                    return 1
                    
                else:
                    return floor
            else:
                return 0
            
        except Exception as err:
            return 0
        
        
    @property
    def getTitle(self):                          # OK
        return self.__title
    
    
    @property
    def getDate(self):
        d = self.__getTime.split(' ')[0]
        fmt = '%Y-%m-%d'
        return time.strptime(d, fmt)
    
    # '樓中樓', '公寓', '別墅', '電梯大樓', '透天厝', '無電梯公寓',  '廠房'
    @property
    def getLabel(self):
        ttype = self.__main['型態']
        if '店面' in ttype:
            return 'S'
        
        if '辦公' in ttype or '住辦' in ttype:
            return 'O'
        
        if ttype in ['樓中樓', '公寓', '別墅', '電梯大樓', '透天厝', '無電梯公寓']:
            return 'H'
        
        if ttype in ['廠房']:
            return 'F'
        else:
            pass

    
    @property
    def getPattern(self):                        # OK
        if self.__main.get('建物格局'):
            return self.__main['建物格局']
        return ''
    
    
    @property
    def getAddress(self):                        # OK
        if self.__main['地址']:
            return self.__main['地址'].replace('　','')
        return ''
    
    
    @property
    def getRent(self):                       # 需要大量debug
        reg = re.compile(r'([0-9]{1,3},?[0-9]{3})+')
        if reg.search(self.__main['租金']):
            mo = reg.match(self.__main['租金'])
            return int(mo.group().replace(',', ''))
        return -1
    
    
    @property
    def getLat(self):                            # OK
        if self.__main.get('lat'):
            return self.__main['lat']
        else:
            return 0
        
    
    @property
    def getLng(self):                            # OK
        if self.__main.get('lng'):
            return self.__main['lng']
        else:
            return 0
    
    
    @property
    def getSex(self):                            # 需要大量debug
        if self.__detail.get('性別限制'):
            if '男女皆可' in self.__detail['性別限制']:
                return 'B'
            elif '限男生' in self.__detail['性別限制']:
                return 'M'
            elif '限女生' in self.__detail['性別限制']:
                return 'F'
        return ''

    @property
    def getSpace(self):                          # 需要大量debug
        if self.__main.get('坪數'):
            mo = re.match(r'\d+(.[\d]+)?', self.__main['坪數'])
            return float(mo.group())
        else:
            return -1
    
    @property
    def getSmoke(self):                          # 需要大量debug
        if self.__detail.get('生活約定'):
            if '抽菸不可' in self.__detail['生活約定']:
                return 'N'
            elif '抽菸可' in self.__detail['生活約定']:
                return 'Y'
        else:
            return ''
    
    @property
    def getPet(self):                            # 需要大量debug
        if self.__detail.get('養寵物'):
            if '不可' in self.__detail['養寵物']:
                return 'N'
            elif '可' in self.__detail['養寵物']:
                return 'Y'
            else:
                return ''
        else:
            return ''
    
    @property
    def getCook(self):                           # 需要大量debug
        if self.__detail.get('開伙'):
            if '不可' in self.__detail['開伙']:
                return 'N'
            elif '可' in self.__detail['開伙']:
                return 'Y'
            else:
                return ''
        else:
            return ''

    
    @property
    def getParking(self):                        # 需要大量debug
        if self.__detail.get('車位'):
            if '無' in self.__detail['車位']:
                return 'N'
            elif '有' in self.__detail['車位']:
                return 'Y'
            else:
                return ''
        return ''
    
# 永慶房屋

In [16]:
# 好房網_建彰
class Extracter_hf():
    pass

    def __init__(self, url, title, main, detail, getTime):
        self.__url = url
        self.__title = title
        self.__main = main
        self.__detail = detail
        self.__getTime = getTime
        self.__YN = True
        
    @property # 調整的儲存程序
    def getYN(self):
        return self.__YN
        
    @property
    def getUrl(self):                            # OK
        return self.__url
    
    @property
    def getTitle(self):                          # OK
        return self.__title
    
    @property                    # @@
    def getDate(self):
        d = self.__getTime.split(' ')[0]
        fmt = '%Y-%m-%d'
        return time.strptime(d, fmt)
    
    @property
    def getLabel(self):
        return 'H'

    
    @property
    def getFloor(self): # 所在樓層 
        return self.__floor('f')
    
    @property
    def getStories(self): # 總樓層 
        return self.__floor('t')

    def __floor(self, ftype):
        try:
            if self.__main.get('樓層'):
                reg = re.compile(r'-?[\d]+') 
                if '/' in self.__main['樓層']:
                    floorString = self.__main['樓層'].split('/')[0] if ftype == 'f' else self.__main['樓層'].split('/')[1]
                    floor = reg.search(floorString).group()

                elif self.__main['樓層'] == '0樓':
                    floorString = self.__main['樓層']
                    floor = reg.search(floorString).group()
                    
                floor = int(floor)
                if floor == 0:
                    return 1
                else:
                    return floor
            
        except Exception as err:
            return 0
    
    @property
    def getPattern(self):                        # OK
        if self.__main['格局']:
            return self.__main['格局']
        return ''
    
    @property
    def getAddress(self):                        # OK
        if self.__main['地址']:
            return self.__main['地址'].replace('　','')
        return ''
    
    @property
    def getRent(self):                       # 需要大量debug
        reg = re.compile(r'([0-9]{1,3},?[0-9]{3})+')
        if reg.search(self.__main['租金']):
            mo = reg.match(self.__main['租金'])
            return int(mo.group().replace(',', ''))
        return -1
    
    @property
    def getLat(self):                            # OK
        return 0
        
    @property
    def getLng(self):                            # OK
        return 0
    
    @property
    def getSex(self):                            # 需要大量debug
        if self.__detail.get('性別限制'):
            if '男女皆可' in self.__detail['性別限制']:
                return 'B'
            elif '限男生' in self.__detail['性別限制']:
                return 'M'
            elif '限女生' in self.__detail['性別限制']:
                return 'F'
        return ''

    @property
    def getSpace(self):                          # 需要大量debug
        if self.__main.get('坪數'):
            mo = re.match(r'\d+(.[\d]+)?', self.__main['坪數'])
            sp = float(mo.group())
            return sp
        else:
            return -1
    
    @property
    def getSmoke(self):
        return ''
    
    @property
    def getPet(self):                            # 需要大量debug
        if self.__detail.get('寵物'):
            if '不可' in self.__detail['寵物']:
                return 'N'
            elif '可' in self.__detail['寵物']:
                return 'Y'
            else:
                return ''
        else:
            return ''
    
    @property
    def getCook(self):                           # 需要大量debug
        if self.__detail.get('開火'):
            if '不可' in self.__detail['開火']:
                return 'N'
            elif '可' in self.__detail['開火']:
                return 'Y'
            else:
                return ''
        else:
            return ''
    
    @property
    def getParking(self):                        # 需要大量debug
        if self.__detail.get('車位'):
            if self.__detail['車位'] in ['--', '無']:
                return 'N'
            else:
                return 'Y'
        else:
            return ''
#

In [17]:
# 好房網_十趴 + 建彰
class Extracter_hf0():
    pass

    def __init__(self, url, title, address, pattern, floor, rent, sex, space, pet, cook, parking, getTime):
        self.__url = url
        self.__title = title
        self.__address = address
        self.__pattern = pattern
        self.__floor = floor
        self.__rent = rent
        self.__sex = sex
        self.__space = space
        self.__pet = pet
        self.__cook = cook
        self.__parking = parking
        self.__getTime = getTime
        self.__YN = True
        
    @property # 調整的儲存程序
    def getYN(self):
        return self.__YN
        
    @property
    def getUrl(self):                            # OK
        return self.__url
    
    @property
    def getTitle(self):                          # OK
        return self.__title
    
    @property                    # 
    def getDate(self):
        d = self.__getTime.split(' ')[0]
        fmt = '%Y-%m-%d'
        return time.strptime(d, fmt)
    
    @property
    def getLabel(self):
        if self.__pattern == '--房(室)--廳--衛':
            return 'F'
        else:
            return 'H'
    
    @property
    def getFloor(self): # 所在樓層 
        return self.__mfloor('f')
    
    @property
    def getStories(self): # 總樓層 
        return self.__mfloor('t')

    def __mfloor(self, ftype):
        try:
            reg = re.compile(r'-?[\d]+') 
            if '/' in self.__floor:
                floorString = self.__floor.split('/')[0] if ftype == 'f' else self.__floor.split('/')[1]
                floor = reg.search(floorString).group()

            elif self.__floor == '0樓':
                floorString = self.__floor
                floor = reg.search(floorString).group()

            floor = int(floor)
            if floor == 0:
                return 1
            else:
                return floor
            
        except Exception as err:
            return 0
    
    @property
    def getPattern(self):                        # OK
        if self.__pattern:
            return self.__pattern
        return ''
    
    @property
    def getAddress(self):                        # OK
        return self.__address
    
    @property
    def getRent(self):                       # 需要大量debug
        reg = re.compile(r'([0-9]{1,3},?[0-9]{3})+')
        if reg.search(self.__rent):
            mo = reg.match(self.__rent)
            return int(mo.group().replace(',', ''))
        return -1
    
    @property
    def getLat(self):                            # OK
        return 0
        
    @property
    def getLng(self):                            # OK
        return 0
    
    @property
    def getSex(self):                            # 需要大量debug
        if '男女皆可' in self.__sex:
            return 'B'
        elif '限男生' in self.__sex:
            return 'M'
        elif '限女生' in self.__sex:
            return 'F'
        else:
            return ''

    @property
    def getSpace(self):                          # 需要大量debug
        try:
            mo = re.match(r'\d+(.[\d]+)?', self.__space)
            sp = float(mo.group())
            return sp
        except Exception as err:
            return 0

    
    @property
    def getSmoke(self):
        return ''
    
    @property
    def getPet(self):                            # 需要大量debug
        if '不可' in self.__pet:
            return 'N'
        elif '可' in self.__pet:
            return 'Y'
        else:
            return ''
    
    @property
    def getCook(self):                           # 需要大量debug
        if '不可' in self.__cook:
            return 'N'
        elif '可' in self.__cook:
            return 'Y'
        else:
            return ''
    
    @property
    def getParking(self):                        # 需要大量debug
        if self.__parking == '無':
            return 'N'
        elif self.__parking == '有':
            return 'Y'
        else:
            return ''
#

In [18]:
# 住商不動產 逸帆+建彰
class Extracter_lb():
    pass
    
    def __init__(self, url, floor, stories, title, address, pattern, label, rent, lat, lng, sex, space, smoke, pet, cook, parking, updateDate):
        self.__url = url
        self.__floor = floor
        self.__stories = stories
        self.__title = title
        self.__address = address
        self.__pattern = pattern
        self.__label = label
        self.__rent = rent
        self.__lat = lat
        self.__lng = lng
        self.__sex = sex
        self.__space = space
        self.__smoke = smoke
        self.__pet = pet
        self.__cook = cook
        self.__parking = parking
        self.__updateDate = updateDate
        
    @property
    def getUrl(self):                            # OK
        return self.__url
    
    @property
    def getFloor(self):
        return self.__floor
    
    @property
    def getStories(self):
        return self.__stories
    
    @property
    def getTitle(self):                          # OK
        return self.__title
    
    @property
    def getAddress(self):                        # OK
        return self.__address
    
    @property
    def getPattern(self):                        # OK
        return self.__pattern
    
    @property
    def getLabel(self):
        return self.__label

    @property
    def getDate(self):
        return self.__updateDate

    @property
    def getRent(self):
        return self.__rent
    
    @property
    def getLat(self):                            # OK
        return self.__lat

    @property
    def getLng(self):                            # OK
        return self.__lng
    
    @property
    def getSex(self):                            # 
        if self.__sex == None:
            return ''
        return self.__sex
    
    @property
    def getSpace(self):                          #
        if self.__space == None:
            return -1
        return self.__space
    
    @property
    def getSmoke(self):                          # 
        if self.__smoke == None:
            return ''
        return self.__smoke
    
    @property
    def getPet(self):                            # 
        if self.__pet == None:
            return ''
        return self.__pet
    
    @property
    def getCook(self):                           # 
        if self.__cook == None:
            return ''
        return self.__cook
    
    @property
    def getParking(self):                        # 
        if self.__parking == None:
            return ''
        return self.__parking
# 住商不動產

In [19]:
# 591租屋 - 正傑, 建彰
class Extracter_591():
    pass

    def __init__(self, url, title, rent, address, context, pattern, getTime):
        self.__url = url
        self.__title = title
        self.__rent = rent
        self.__address = address[0]
        self.__context = context
        self.__pattern = pattern
        self.__getTime = getTime
        
    @property
    def getUrl(self):                            # OK
        return self.__url
    
    @property
    def getFloor(self): # 所在樓層 
        return self.__floor('f')
    
    def __floor(self, ftype):
        try:
            if self.__pattern.get('樓層'):
                reg = re.compile(r'-?[\d]+') 
                
                if self.__pattern['樓層'] == None:
                    return 0
                
                elif '/' in self.__pattern['樓層']:
                    floorString = self.__pattern['樓層'].split('/')[0] if ftype == 'f' else self.__pattern['樓層'].split('/')[1]
                    floor = reg.search(floorString).group()
                    
                elif self.__pattern['樓層'] == '0樓':
                    floorString = self.__pattern['樓層']
                    floor = reg.search(floorString).group()
                    
                floor = int(floor)
                
                if floor == 0:
                    return 1
                    
                else:
                    return floor
            else:
                return 0
            
        except Exception as err:
            return 0
    
    @property
    def getStories(self): # 總樓層 
        return self.__floor('t')
    
    @property
    def getTitle(self):                          # OK
        return self.__title
    
    
    @property
    def getDate(self):
        return self.__getTime

    
    @property
    def getLabel(self):
        if self.__pattern.get('現況'):
            if self.__pattern['現況'] in ['獨立套房', '雅房', '分租套房', '整層住家']:
                return 'H'
            elif self.__pattern['現況'] == '車位':
                return 'F'
            elif self.__context.get('法定用途'):
                if self.__context['法定用途'] == '住家用':
                    return 'H'
        
        if self.__pattern.get('型態'):
            if self.__pattern['型態'] in ['住宅大樓', '別墅', '華廈', '透天厝', '公寓', '電梯大樓']:
                return 'H'
            elif self.__pattern['型態'] in ['倉庫', '工廠', '廠辦', '工廠', '農舍']:
                return 'F'
            elif self.__pattern['型態'] in ['店面（店鋪）']:
                return 'S'
            elif self.__pattern['型態'] in ['辦公商業大樓']:
                return 'O'
        
        
        if '倉儲' in self.__title:
            return 'F'
        elif '工作室' in self.__title or '辦公' in self.__title:
            return 'O'
        elif '居住' in self.__title or '生活' in self.__title or '社區' in self.__title or '裝潢' in self.__title:
            return 'H'
        
#     格局 :  3房2廳2衛2陽台
    
    @property
    def getPattern(self):                        # OK 
        if self.__pattern.get('格局'): 
            return self.__pattern['格局']
        elif self.__pattern.get('現況'):
            return self.__pattern['現況']
        else:
            return ''
    
    @property
    def getAddress(self):                        # OK
        return self.__address
    
    @property
    def getRent(self):                     
        reg = re.compile(r'([0-9]{1,3},?[0-9]{3})+')
        if reg.search(self.__rent):
            mo = reg.match(self.__rent)
            return int(mo.group().replace(',', ''))
        return -1
    
    @property
    def getLat(self):                            # OK
        return 0
#         return self.__main['latlng'].split(',')[1]
        
    
    @property
    def getLng(self):                            # OK
        return 0
        #return self.__main['latlng'].split(',')[0]
    
    @property
    def getSex(self):                            # 需要大量debug
        if self.__context.get('性別要求'):
            if '男女生皆可' in self.__context['性別要求']:
                return 'B'
            elif '男生' in self.__context['性別要求']:
                return 'M'
            elif '女生' in self.__context['性別要求']:
                return 'F'
        return ''

    @property
    def getSpace(self):                          # 需要大量debug
        if self.__pattern.get('坪數'):
            mo = re.match(r'\d+(.[\d]+)?', self.__pattern['坪數'])
            return float(mo.group())
        else:
            return -1
    
    @property
    def getSmoke(self):                          # 需要大量debug
        return ''
    
    @property
    def getPet(self):                            # 需要大量debug
        if self.__context.get('養寵物'):
            if '不可以' in self.__context['養寵物']:
                return 'N'
            elif '可以' in self.__context['養寵物']:
                return 'Y'
        else:
            return ''
    
    @property
    def getCook(self):                           # 需要大量debug
        if self.__context.get('開伙'):
            if '不可以' in self.__context['開伙']:
                return 'N'
            elif '可以' in self.__context['開伙']:
                return 'Y'
        else:
            return ''
        
    @property
    def getParking(self):                        # 需要大量debug
        if self.__context.get('車 位'):
            if '無' == self.__context['車 位']:
                return 'N'
            elif self.__context['車 位'] in ['平面式停車位，已含租金內', '平面式停車位，費用另計', '機械式停車位，已含租金內', '機械式停車位，費用另計']:
                return 'Y'
        else:
            return ''
# 591

In [20]:
# 太平洋、21世紀 柏皓+建彰 @@
class Extracter_csv():
    pass

    def __init__(self, src):
        self.__src = src
        self.__YN = True
        
    @property
    def getUrl(self):
        return self.__src[0]
    
    @property
    def getTitle(self):
        return self.__src[1]
    
    @property
    def getAddress(self):
        if '宜蘭' in self.__src[2] or '花蓮' in self.__src[2] or '台東' in self.__src[2] or '屏東' in self.__src[2] or '高雄' in self.__src[2] or '台南' in self.__src[2] or '嘉義' in self.__src[2] or '雲林' in self.__src[2] or '彰化' in self.__src[2] or '台中' in self.__src[2] or '苗栗' in self.__src[2] or '南投' in self.__src[2] or '澎湖' in self.__src[2] or '金門' in self.__src[2]:
            self.__YN = False
        return self.__src[2]
    
    @property
    def getPattern(self):
        if self.__src[3] == 'NULL':
            return ''
        else:
            return self.__src[3]

    @property
    def getFloor(self):
        return self.__src[4]
    
    @property
    def getLabel(self):
        if '三角' in self.__src[1] or '店面' in self.__src[1]:
            return 'S'
        elif '農地' in self.__src[1] or '土地' in self.__src[1] or '倉儲' in self.__src[1]:
            return 'F'
        elif '住辦' in self.__src[1]:
            return 'O'
        else:
            return self.__src[5] if self.__src[5] != 'NULL' else 'H'
    
    @property
    def getRent(self):
        return int(self.__src[6].replace(',', ''))
    
    @property
    def getLat(self):
        return round(float(self.__src[7]), 10)
    
    @property
    def getLng(self):
        return round(float(self.__src[8]), 10)
    
    @property
    def getSex(self):
        if self.__src[9] == 'NULL':
            return ''
        else:
            return self.__src[9]
    
    @property
    def getSpace(self):
        if self.__src[10] == 'NULL':
            return 0
        else:
            return round(float(self.__src[10].replace(',', '')), 2)
    
    @property
    def getSmoke(self):
        if self.__src[11] == 'NULL':
            return ''
        else:
            return self.__src[11]
    
    @property
    def getPet(self):
        if self.__src[12] == 'NULL':
            return ''
        else:
            return self.__src[12]
    
    @property
    def getCook(self):
        if self.__src[13] == 'NULL':
            return ''
        else:
            return self.__src[13]
    
    @property
    def getParking(self):
        if self.__src[14] == 'NULL':
            return ''
        else:
            return self.__src[14]
    
    @property
    def getDate(self):
        d = self.__src[15].split(' ')[0]
        fmt = '%Y-%m-%d'
        return time.strptime(d, fmt)

    @property
    def getStories(self):
        return 0
# pac + 21

In [21]:
# 寫出至json, 準備匯入mongoDB
def append2mongoJSON(fileName, obj):
    obj = str(obj)
    with open(fileName, 'a', encoding='utf8') as f:
        f.write(obj + "\n")
#

# 產生可逐筆 append 到 mongoFormatJSON 的物件
def writeMongoFormat(houseObj):
    obj = {}
    obj['url'] = houseObj.getUrl
    obj['title'] = houseObj.getTitle
    obj['address'] = houseObj.getAddress
    obj['pattern'] = houseObj.getPattern
    return obj
#

In [22]:
# 資料異常調整
def adj(dbhost, user, passwd):
    db = pymysql.connect(dbhost, user, passwd, 'house')
    c = db.cursor()
    
    # to do list:
    updQrys = []
    updQrys.append("UPDATE rental SET lat = 0, lng = 0 WHERE lat > 9999 OR lng > 9999;")        # 1. 更改 lat, lng不合理者 -> 0
    # updQrys.append("DELETE FROM rental WHERE lat > 25.298")                                     # 2. 刪除經緯度不合理者
    # updQrys.append("DELETE FROM rental WHERE lat = 0")                # 3. 刪除經緯度不合理者(※ 發表前執行!!!)
#     updQrys.append("DELETE FROM rental WHERE rent < 1000")

    for qry in updQrys:
        c.execute(qry)
        db.commit()
        
    c.close()
    db.close()
#

In [23]:
# 逐筆讀取送入解析器
def mainProg(target):
    os.chdir(r'D:/bbproj/house')
#     os.chdir('../')
    cfgini = 'params/house.ini'
    dbhost, user, passwd = getConfig(cfgini)
#     fileName = 'mongo_' + target[0:2] + '.txt'
#     os.remove(fileName) if os.path.exists(fileName) else None
    
    if target in ['sy.db', 'jc.db', 'yc.db', 'hf.db']:
        tbl = readDB(target)
        urls, titles, mains, details, getTimes = splitColumnFromDataSet(tbl)
        
        for i in range(0, len(urls)):
            url = urls[i]
            title = titles[i]
            main = eval(mains[i])
            detail = eval(details[i])
            getTime = getTimes[i]

            if target == 'sy.db':
                houseObj = Extracter_sy(url, title, main, detail, getTime)  # 信義房屋
            elif target == 'jc.db':
                houseObj = Extracter_jc(url, title, main, detail, getTime) # 永慶房屋
            elif target == 'yc.db':
                houseObj = Extracter_yc(url, title, main, detail, getTime) # 永慶房屋
            elif target =='hf.db':
                houseObj = Extracter_hf(url, title, main, detail, getTime) # 好房網
                print(url)
                
            # print(h.getUrl); print(h.getTitle); print(h.getAddress); print(h.getPattern); print(str(h.getFloor)); print(str(h.getStories)); print(h.getLabel); print(str(h.getRent)); print(h.getLat); print(h.getLng); print(h.getSex); print(str(h.getSpace)); print(h.getSmoke); print(h.getPet); print(h.getCook); print(h.getParking); print(h.getDate);sys.exit()
            
            upsertHouse(houseObj, dbhost, user, passwd)
#             obj = writeMongoFormat(houseObj)
#             qq = json.dumps(obj, ensure_ascii=False)
#             append2mongoJSON(fileName, qq)
            
    elif target == '591.json': 
        tbl = readDB_json(target)
        urls, rents, addresses, contexts, titles, patterns, getTimes= splitColumnFromDataSet_591(tbl)
        
        for i in range(0, len(urls)):
            url = urls[i];           rent = rents[i];          address = addresses[i];
            context = contexts[i];   title = titles[i];        pattern = patterns[i];     getTime = getTimes[i]
            
            houseObj = Extracter_591(url, title, rent, address, context, pattern, getTime)
            upsertHouse(houseObj, dbhost, user, passwd)
    elif target in ['21.csv', 'pac.csv']:
        tbl = readDB_csv(target)

        for r in tbl:
            houseObj = Extracter_csv(r)
            
            upsertHouse(houseObj, dbhost, user, passwd)

    elif target in ['hf01.json', 'hf02.json', 'hf03.json', 'hf04.json', 'hf05.json', 'hf06.json']:  # 好房網 10%版本
        tbl = readDB_json(target)
        urls, titles, addresss, patterns, floors, rents, sexs, spaces, pets, cooks, parkings, getTimes = splitColumnFromDataSet_hf(tbl)

        for i in range(0, len(urls)):
            url = urls[i]
            title = titles[i]
            address = addresss[i]
            pattern = patterns[i]
            floor = floors[i]
            rent = rents[i]
            sex = sexs[i]
            space = spaces[i]
            pet = pets[i]
            cook = cooks[i]
            parking = parkings[i]
            getTime = getTimes[i]
            
            houseObj = Extracter_hf0(url, title, address, pattern, floor, rent, sex, space, pet, cook, parking, getTime)
            upsertHouse(houseObj, dbhost, user, passwd)
            
    elif target == 'lb.db':
        tbl = readDB_lb(target)
        urls, titles, addresss, patterns, labels, rents, lats, lngs, sexs, spaces, smokes, pets, cooks, parkings, updateDates, storiess, floors = splitColumnFromDataSet_lb(tbl)

        for i in range(0, len(urls)):
            url = urls[i];            title = titles[i];  address = addresss[i];  pattern = patterns[i];  label = labels[i];           floor = floors[i];
            rent = rents[i];          lat = lats[i];      lng = lngs[i];          sex = sexs[i];          space = spaces[i];           stories = storiess[i]
            smoke = smokes[i];        pet = pets[i];      cook = cooks[i];        parking = parkings[i];  updateDate = updateDates[i]

            houseObj = Extracter_lb(url, floor, stories, title, address, pattern, label, rent, lat, lng, sex, space, smoke, pet, cook, parking, updateDate)

            upsertHouse(houseObj, dbhost, user, passwd)
            
    adj(dbhost, user, passwd)
#

In [25]:
def gogo():
    targets = ['sy.db', 'jc.db', 'yc.db']
#     targets = ['hf01.json', 'hf02.json', 'hf03.json', 'hf04.json', 'hf05.json', 'hf06.json']
#     targets = ['hf02.json']
#     targets = ['hf.db']
#     targets = ['lb.db']
    for target in targets:
        mainProg(target)
#

In [26]:
gogo()

c:\users\java\appdata\local\programs\python\python35\lib\site-packages\pymysql\cursors.py:166: Warning: (1265, "Data truncated for column 'lat' at row 1")
  result = self._query(query)
c:\users\java\appdata\local\programs\python\python35\lib\site-packages\pymysql\cursors.py:166: Warning: (1265, "Data truncated for column 'lng' at row 1")
  result = self._query(query)
